# 블로그 컨텐츠 수집

## #01. 웹 페이지 컨텐츠 수집 개요

### [1] 웹 페이지 수집 기술 종류

#### (1) 웹 스크랩핑

하나의 웹 페이지에서 컨텐츠 일부를 수집하는 기술

#### (2) 웹 크롤링

하나의 웹 페이지에 연결된 다른 페이지까지 순차적으로 수집하는 기술

크롤링을 수행하는 소프트웨어를 **크롤러** 라고 한다.

대부분의 검색엔진이 크롤러에 해당한다.

> 웹 페이지로부터 컨텐츠를 수집하기 위해서는 웹 페이지의 코드 구조를 사람이 파악하는 스킬이 중요하다.

**html형식을 긁어오는 형태이기때문에 웹페이지가 개편되면 이전에 사용했던 코드는 재사용이 불가하다**

### [2] 크롬 개발자 도구

크롬 브라우저에서 `F12`키를 누르면 표시되는 웹페이지 분석 도구

![그림](devtools.png)

## #02. 준비과정
### [1] 패키지 참조


In [ ]:
import requests
from bs4 import BeautifulSoup
import datetime as dt

### [2] 접속할 데이터의 URL

In [ ]:
url = "https://blog.hossam.kr"

## #03. 데이터 요청하기
### [1] 세션요청

In [ ]:
session = requests.Session()

session.headers.update({
    "Referer": "",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
})

### [2] 웹 페이지 소스코드 가져오기

웹 페이지를 구성하는 HTML 소스코드를 `BeautifulSoup` 클래스 객체로 변환한다

In [ ]:
try:
    r = session.get(url)
    
    if r.status_code != 200:
        msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
        raise Exception(msg)
except Exception as e:
    print("접속에 실패했습니다.")
    print(e)


r.encoding = "utf-8"
soup = BeautifulSoup(r.text)


## #04. 데이터 추출하기

### [1] 원하는 부분 지정하는 전략

1. 고유한 요소인 경우 `id` 속성을 찾는다.
2. `id`가 없을 경우 혹은 복수 요소인 경우 `class` 속성을 찾는다.
3. 둘 다 없을 경우 HTML 태그를 몇 단계 거슬러 올라가 `id` 나 `class`를 찾는다.
   - `id`와 `class`는 고유 항목인지 복수 항목인지에 따라 결정한다.
   - `class`가 반드시 복수 항목을 의미하는 것은 아니다.
4. 태그 단계를 거슬러 올라간 경우 원하는 요소까지 자식/자손 선택자로 접근한다.(자손 선택자 권장)

### [2] 글 묶음 가져오기

soup객체의 `select()`메서드에 CSS선택자를 파라미터로 전달하여 원하는 부분을 추출한다.

리턴 결과는 항상 리스트 형식이다.

> 추출한 결과가 몇 개인지 항상 체크해야 한다.

In [ ]:
post = soup.select('.post')

print("추출한 컨텐츠의 수 :", len(post))


### [3] 개별 글에 대한 처리

In [90]:
mydata =[]
for i, v in enumerate(post):
    # print(v)

    # 하나의 글 안에서 제목 영역을 찾는다.
    entryTitle = v.select(".entry-title a")
    # print(entryTitle)

    # 추출된 요소가 각 게시글 안에서 하나만 존재하므로 `0`번째 원소에 직접 접근한다.
    title = entryTitle[0].text.strip()
    # print(title)

    # 클릭시 이동할 페이지의 주소
    if 'href' in entryTitle[0].attrs:
        href = entryTitle[0].attrs['href']

    # 같은 페이지 내에서 주소 이동할 경우 도메인 생략가능
        
    # 수집된 주소에 도메인이 없다면 덧붙여준다
    if url not in href:
        href = url+href

    else:
        href = None
    # print(href)

    # 작성일
    published = v.select(".published")
    # print(published)
    datetime = published[0].attrs['datetime']
    # print(datetime)
    
    # 요약글
    entryContent = v.select(".entry-content p")
    # print(entryContent)

    # 마지막의 `more` 버튼은 제거한다.
    # entryContent = entryContent[:-1] # python적 접근
    # print(entryContent)

    entryContent = v.select(".entry-content p:not(.read-more)") # CSS적 접근
    # print(entryContent)

    text = ''
    for j, e in enumerate(entryContent): # 2개 이상인 경우도 있으므로 반복문
        text += " " +e.text.strip() if j>0 else e.text.strip()
    # print(text)

    # 태그
    tagLinks = v.select(".tag-links a")
    # print(tagLinks)

    
    for j,e in enumerate(tagLinks):
        tagLinks[j] = e.text.strip()
    
    tags = ','.join(tagLinks)
    
    # 수집된 정보를 하나의 딕셔너리로 묶는다 => 처리방법은 다르게해도 상관 없음
    mydict = {
        "title":title,
        "href":href,
        "datetime":datetime,
        "text":text,
        "tags":tags
    }
    mydata.append(mydict)


{'href': '/2023/08/21/Toy-%EC%84%9C%EB%B2%84_%EA%B0%80%EB%8F%99_%EC%84%A4%EC%A0%95_%EC%88%98%EC%A0%95/', 'rel': ['bookmark'], 'title': 'Permanent Link to /2023/08/21/Toy-%EC%84%9C%EB%B2%84_%EA%B0%80%EB%8F%99_%EC%84%A4%EC%A0%95_%EC%88%98%EC%A0%95/'}
{'href': '/2023/08/20/Toy-Next.js+Express-%EB%B3%91%ED%95%A9/', 'rel': ['bookmark'], 'title': 'Permanent Link to /2023/08/20/Toy-Next.js+Express-%EB%B3%91%ED%95%A9/'}
{'href': '/2023/08/17/Toy-%ED%86%A0%EC%9D%B4%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8-%EC%8B%9C%EC%9E%91/', 'rel': ['bookmark'], 'title': 'Permanent Link to /2023/08/17/Toy-%ED%86%A0%EC%9D%B4%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8-%EC%8B%9C%EC%9E%91/'}
{'href': '/2023/07/13/Mac-OpenJDK-%EC%84%A4%EC%B9%98/', 'rel': ['bookmark'], 'title': 'Permanent Link to /2023/07/13/Mac-OpenJDK-%EC%84%A4%EC%B9%98/'}
{'href': '/2023/07/12/Mac-%EA%B0%9C%EB%B0%9C%ED%99%98%EA%B2%BD%EA%B5%AC%EC%84%B1-%ED%95%84%EC%88%98%EC%9A%94%EC%86%8C/', 'rel': ['bookmark'], 'title': 'Permanent Link to /2023/07/12/Mac-%E

### [4] 수집 결과 저장하기

In [ ]:
fname = dt.datetime.now().strftime("블로그_글_수집_%y%m%d_%H%M%S.csv")

with open(fname, 'w', encoding='utf-8') as f:
    for i,v in enumerate(mydata):
        if i ==0 : 
            title = f'{",".join(v.keys())}\n'
            # print(title)
            f.write(title)
        
        content = list(v.values())

        # print(content)
        # csv에 저장하기 위해서
        # 1) 각 컨텐츠 안에 포함된 쌍따옴표는 역슬래시로 묶어준다
        # 2) 각 컨텐츠를 쌍따옴표로 묶어준다.
        for j,w in enumerate(content):
            content[j] = f'"{w.replace('"',r'\"')}"'
        # print(content)
        f.write(f'{",".join(content)}\n')